In [19]:
import requests
import json
import pandas as pd
import numpy as np
import pymysql
import os

TMDB API에서 영화id를 이용하여 각 영화의 cast정보를 가져올 수 있음. 

"https://api.themoviedb.org/3/movie/{movie_id}?api_key=36927ad1d2817ff0fa31947e47d186fd&append_to_response=credits"

In [20]:
# 영화id가 512195인 cast정보 크롤링

cast_url="https://api.themoviedb.org/3/movie/512195?api_key=36927ad1d2817ff0fa31947e47d186fd&append_to_response=credits"
response = requests.get(cast_url)
json_object = response.content
json.loads(json_object)

# 항목 credits아래 cast정보가 있음. 'credits': {'cast': [{'adult': False,....

{'adult': False,
 'backdrop_path': '/5uVhMGsps81CN0S4U9NF0Z4tytG.jpg',
 'belongs_to_collection': None,
 'budget': 200000000,
 'genres': [{'id': 28, 'name': 'Action'},
  {'id': 35, 'name': 'Comedy'},
  {'id': 80, 'name': 'Crime'},
  {'id': 53, 'name': 'Thriller'}],
 'homepage': 'https://www.netflix.com/us/title/81161626',
 'id': 512195,
 'imdb_id': 'tt7991608',
 'original_language': 'en',
 'original_title': 'Red Notice',
 'overview': "An Interpol-issued Red Notice is a global alert to hunt and capture the world's most wanted. But when a daring heist brings together the FBI's top profiler and two rival criminals, there's no telling what will happen.",
 'popularity': 4404.247,
 'poster_path': '/wdE6ewaKZHr62bLqCn7A2DiGShm.jpg',
 'production_companies': [{'id': 34081,
   'logo_path': None,
   'name': 'Flynn Picture Company',
   'origin_country': 'US'},
  {'id': 73669,
   'logo_path': '/7tmSGstK3KwgcDIuBYLTAayjit9.png',
   'name': 'Seven Bucks Productions',
   'origin_country': 'US'},
  {'i

In [21]:
# cast 정보만 가져오기
movie_data=json.loads(json_object)
movie_data["credits"]["cast"]

[{'adult': False,
  'gender': 2,
  'id': 18918,
  'known_for_department': 'Acting',
  'name': 'Dwayne Johnson',
  'original_name': 'Dwayne Johnson',
  'popularity': 33.478,
  'profile_path': '/kuqFzlYMc2IrsOyPznMd1FroeGq.jpg',
  'cast_id': 2,
  'character': 'John Hartley',
  'credit_id': '5aac343892514116b7004626',
  'order': 0},
 {'adult': False,
  'gender': 2,
  'id': 10859,
  'known_for_department': 'Acting',
  'name': 'Ryan Reynolds',
  'original_name': 'Ryan Reynolds',
  'popularity': 43.06,
  'profile_path': '/4SYTH5FdB0dAORV98Nwg3llgVnY.jpg',
  'cast_id': 13,
  'character': 'Nolan Booth',
  'credit_id': '5d23cf7069d2800010011045',
  'order': 1},
 {'adult': False,
  'gender': 1,
  'id': 90633,
  'known_for_department': 'Acting',
  'name': 'Gal Gadot',
  'original_name': 'Gal Gadot',
  'popularity': 27.976,
  'profile_path': '/AbXKtWQwuDiwhoQLh34VRglwuBE.jpg',
  'cast_id': 9,
  'character': 'Sarah Black / The Bishop',
  'credit_id': '5b1ea4290e0a261010008a2d',
  'order': 2},
 {'ad

In [22]:
# 영화id를 사용하기 위해 영화정보 파일가져오기 
tmdb_movie_list = pd.read_csv('../data-files/total_tmdbmovielist.csv',index_col=0) 

C:\Users\user\.conda\envs\ykseo\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (1,4,7,12,17,23) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [23]:
# 영화id를 이용하여 cast 정보 crawling  (전체영화, 오래걸림)

tmdb_movie_id_list = tmdb_movie_list[["id"]]

all_cast_list = []
error_list = []
for idx,  movie_id in enumerate(tmdb_movie_id_list["id"].values):  # movie_id 목록을 기준으로 인덱스와 movie_id를 반환
    try :
        cast_url="https://api.themoviedb.org/3/movie/{0}?api_key=36927ad1d2817ff0fa31947e47d186fd&append_to_response=credits"
        response = requests.get(cast_url.format(movie_id))
        json_object = response.content
        movie_data = json.loads(json_object)
        cast_list = movie_data["credits"]["cast"]
        for cast in cast_list:                                      
            cast["movie_id"] = movie_id
            all_cast_list.append(cast)

    except:
        error_list.append(movie_id)

    if idx == 100:
        break

In [24]:
# 현재 상영 중인 영화
now_playing_movies=pd.read_csv("../data-files/now_playing_movies.csv", index_col=0)

In [25]:
now_playing_movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200 entries, 0 to 199
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   adult              200 non-null    bool   
 1   backdrop_path      200 non-null    object 
 2   genre_ids          200 non-null    object 
 3   id                 200 non-null    int64  
 4   original_language  200 non-null    object 
 5   original_title     200 non-null    object 
 6   overview           160 non-null    object 
 7   popularity         200 non-null    float64
 8   poster_path        200 non-null    object 
 9   release_date       200 non-null    object 
 10  title              200 non-null    object 
 11  video              200 non-null    bool   
 12  vote_average       200 non-null    float64
 13  vote_count         200 non-null    int64  
dtypes: bool(2), float64(2), int64(2), object(8)
memory usage: 20.7+ KB


In [26]:
now_playing_movies_id_list = now_playing_movies[["id"]]
now_playing_movies_id_list

,id
0,580489
1,512195
2,585245
3,566525
4,568124
...,...
195,116322
196,817566
197,8373
198,258489


In [27]:
now_playing_movies_id_list = now_playing_movies[["id"]]

all_cast_list = []
error_list = []
for idx,  movie_id in enumerate(now_playing_movies_id_list["id"].values):  # movie_id 목록을 기준으로 인덱스와 movie_id를 반환
    try :
        cast_url="https://api.themoviedb.org/3/movie/{0}?api_key=36927ad1d2817ff0fa31947e47d186fd&append_to_response=credits"
        response = requests.get(cast_url.format(movie_id))
        json_object = response.content
        movie_data = json.loads(json_object)
        cast_list = movie_data["credits"]["cast"]
        for cast in cast_list:                                      
            cast["movie_id"] = movie_id
            all_cast_list.append(cast)

    except:
        error_list.append(movie_id)

    if idx == 100:
        break

In [35]:
# cast 정보 csv파일로 저장 
casts=pd.DataFrame(all_cast_list)
casts


,adult,gender,id,known_for_department,name,original_name,popularity,profile_path,cast_id,character,credit_id,order,movie_id
0,False,2,2524,Acting,Tom Hardy,Tom Hardy,44.119,/9bXUMLOBCcnhcUGtfw0pdqbvpiH.jpg,1,Eddie Brock / Venom,5c5b3ebfc3a3683cc6885550,0,580489
1,False,2,57755,Acting,Woody Harrelson,Woody Harrelson,22.604,/igxYDQBbTEdAqaJxaW6ffqswmUU.jpg,7,Cletus Kasady / Carnage,5c86bc069251410d3616ff8e,1,580489
2,False,1,1812,Acting,Michelle Williams,Michelle Williams,11.940,/mfgKyigaQLI0NtZsaJRVlbUiMdX.jpg,9,Anne Weying,5d4b75510021340013269639,2,580489
3,False,1,2038,Acting,Naomie Harris,Naomie Harris,15.663,/pDFs4gSeKSyIF8PditvtHqAq687.jpg,14,Frances Louise Barrison / Shriek,5daa557cb2681f0014915771,3,580489
4,False,2,81316,Acting,Reid Scott,Reid Scott,4.425,/kBAeDUDA7XJRXFLGNALlpE5d3lA.jpg,13,Dr. Dan Lewis,5d8d512979b3d4001f894708,4,580489
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2920,False,0,2946927,Acting,Abel Arias,Abel Arias,0.600,None,32,Tito Turtle (voice) / Ozzie Ostrich (voice),600a8096d55c3d00406d21d4,24,643586
2921,False,2,203236,Acting,Mark Gagliardi,Mark Gagliardi,0.600,/buy4gglZAl9shZPst8Mu0KfLW40.jpg,33,Gus Gorilla (voice),600a80a8e04d8a003ec40397,25,643586
2922,False,0,2946921,Acting,Luke James Donnelly,Luke James Donnelly,0.600,None,28,Arty Alligator (voice),600a7f6441ad8d003e92073b,26,643586
2923,False,0,2946922,Acting,Waverly Dooley,Waverly Dooley,0.600,None,29,Siren Sara (voice),600a7f79cf9ba3003e838bab,27,643586


In [36]:
casts.to_csv("../data-files/cast.csv", header=True, index=True)

In [37]:
# cast정보 DB(mysql)에 저장 
import pymysql

conn = pymysql.connect(host="localhost",
                       database="finalteam3",
                       user="kdigital",
                       password="mysql",
                       charset="utf8")

cursor = conn.cursor()
#cursor.execute('DELETE FROM actor')
error_list = []
for cast in casts.values:
    try:
        sql= "INSERT INTO actor VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
        cursor.execute(sql,list(cast)) 
    except:
        error_list.append(cast)

conn.commit() # 이전에 실행된 SQL 결과를 확정
cursor.close()
conn.close()